In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import (Input, Conv2D, Dense, MaxPool2D, Dropout,
                          Concatenate, Flatten, GlobalMaxPooling2D, Subtract)
from keras.losses import sparse_categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.applications.densenet import DenseNet169
from keras.applications.inception_v3 import InceptionV3
from keras.regularizers import l2
import numpy as np
import keras.backend as K
import h5py
from EgoCNN import EgoCNN, EgoDense, EgoInception, EgoResNet
from ResNet34Bottom import ResNet34Bottom

Using TensorFlow backend.


In [2]:
model = ResNet34Bottom(inputShape=(224,224,3))

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 56, 56, 64)   0           activation_1[0][0]               
__________________________________________________________________________________________________
conv2d_2 (

In [ ]:
f = h5py.File('extraimages_224.hdf5') 
ego_cnn = EgoResNet(in_size=(224,224,3), out_size=(1,1,1), handle=f['train_img'])
model = ego_cnn.model
model.compile(optimizer='adam',
              loss='mse')
train_generator = ego_cnn.batch_generator(f['train_img'])
#train_generator = ego_cnn.batch_generator()
#model.fit_generator(train_generator,
#                    steps_per_epoch=1,
#                    epochs=2)

In [ ]:
for item in ego_cnn.batch_generator(f['train_img']):
    print(np.random.randint(4))
    plt.imshow(item[0]['input0'][0])
    break

In [ ]:
plt.imshow(item[0]['input1'][0])

In [ ]:
model.fit_generator(train_generator,
                    steps_per_epoch=1,
                    epochs=2)

In [ ]:
im = next(train_generator)

In [ ]:
im[1]['output'].shape